# Reccomender system

#### Once we have our data base in vectorised form we can also provide a way to provide articles that are similar to the one that was given as an input. This can be useful for authors at it might point out articles that are related with their work and that they were not aware of. As a first attempt to solve this type of problem we can use cosine similarity.

#### The code contained here is basically the same contained in the module model.py

In [1]:
import pickle
import os
import re
import numpy as np
from random import shuffle
import shutil
from  urllib.request import urlopen


from utils import Config, read_clean

from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
with open(Config.tfidf,'rb') as file:
    tfidf=pickle.load(file)
with open(Config.vectorized_articles,'rb') as file:
    vectorized_articles=pickle.load(file)

In [3]:
X=vectorized_articles['X']

In [4]:
#Function that return the articles in the database X that are closer to the given article x based on cos similarity
def find_similar(X,x,how_many):
    cos_similarity=x.dot(X.transpose())
    cos_similarity=np.asarray(cos_similarity.todense())
    return np.argsort(-cos_similarity)[:,:how_many]

#### We can see an example on how this works on the first two articles of the database.

In [5]:
print(vectorized_articles['articles'][find_similar(X,X[0:2,:],10)])
print(vectorized_articles['links'][find_similar(X,X[0:2,:],10)])

[['2003.01771' '1902.08464' '1807.09515' '1901.07357' '1910.00583'
  '0910.5001' '1904.01053' '2002.06177' '1711.00678' '1911.01756']
 ['1908.02651' '2001.01266' '2002.08316' '1908.02640' '1907.00271'
  '1902.06655' '1906.00478' '1911.03282' '1911.08356' '1906.09380']]
[['http://arxiv.org/pdf/2003.01771v1' 'http://arxiv.org/pdf/1902.08464v3'
  'http://arxiv.org/pdf/1807.09515v1' 'http://arxiv.org/pdf/1901.07357v1'
  'http://arxiv.org/pdf/1910.00583v2' 'http://arxiv.org/pdf/0910.5001v1'
  'http://arxiv.org/pdf/1904.01053v1' 'http://arxiv.org/pdf/2002.06177v3'
  'http://arxiv.org/pdf/1711.00678v1' 'http://arxiv.org/pdf/1911.01756v2']
 ['http://arxiv.org/pdf/1908.02651v3' 'http://arxiv.org/pdf/2001.01266v2'
  'http://arxiv.org/pdf/2002.08316v1' 'http://arxiv.org/pdf/1908.02640v1'
  'http://arxiv.org/pdf/1907.00271v1' 'http://arxiv.org/pdf/1902.06655v1'
  'http://arxiv.org/pdf/1906.00478v3' 'http://arxiv.org/pdf/1911.03282v2'
  'http://arxiv.org/pdf/1911.08356v1' 'http://arxiv.org/pdf/1906

## Evaluation of the reccomender system

#### By looking the articles given by the reccomender system it can be seen that they seem to be very similar to the one given as an input. Anyhow we would like to have a more quantitative measurement on how good our reccomender system is. 
#### In general this is not an easy problem, as we do not know if the reccomended articles are relevant or not, as it would require to read the articles, which for large number of articles is clearly not feasible. One possible approach is then to consider as relevant all the articles that are cited by the paper that we are considering. Since it might be possible that not all the articles cited by artices in the database are also part of the database we will need to download the cited articles
#### Unfortunately most of the time cited articles are not on free acess and this would render it difficult to download them. For this reason we will only consider citations to arXiv articles, which we can easily download. The following approach, therefore, present evident limitations, which could be overcame or at least reduced with more accessible resources. 
#### Another point to be consider is that sometimes not all the articles that are being cited are relevant, these type of complications could be possibly be resolved through some data labeling, wich we will not perform here.

#### Let's therefore start by selecting a set of articles on which we will perform the analysis

In [6]:
with open(Config.metadata_db,'rb') as file:
    metadata_db=pickle.load(file)

txt_labels_train=[]

for article in metadata_db.keys():
    article_path=os.path.join(Config.txt_db,article+'.txt')
    if os.path.isfile(article_path):
        txt_labels_train.append(article)    

In [7]:
txt_labels_shuffle=list(txt_labels_train) #Hard copy for the set of articles
shuffle(txt_labels_shuffle)
#We use the read_clean function as it will remove the citation to the paper itsels
max_articles=7000
idxs=txt_labels_shuffle[:max_articles]

In [8]:
citations_db={}
for idx in idxs:
    with open(os.path.join(Config.txt_db,idx+'.txt'),'r') as file:
        article=file.read()
    citations=re.findall('arXiv:[0-9]+.[0-9]+|arXiv:[a-zA-Z-]+.[0-9]+|arXiv:[a-zA-Z-]+.[0-9]+.[0-9]',article)
    if len(citations)>1:
        citations_db[idx]=citations[1:] # the first citation is generally the paper itself

#### We can now see what the citations look like and save them.

In [9]:
print(citations_db)

{'2001.03896': ['arXiv:1808.00773'], '2001.02756': ['arXiv:1811.03722', 'arXiv:quantph/0209083', 'arXiv:1712.02589', 'arXiv:quantph/0504051'], '1911.12231': ['arXiv:1809.07609', 'arXiv:1706.04702', 'arXiv:1905.05027', 'arXiv:1502.03167', 'arXiv:1412.6980', 'arXiv:1907.10578', 'arXiv:1807.06622', 'arXiv:1904.05921'], '1901.10393': ['arXiv:1909.00598', 'arXiv:1802.07075', 'arXiv:1809.02536', 'arXiv:1210.4034', 'arXiv:1409.2191', 'arXiv:1507.04951'], '1904.11854': ['arXiv:1604.08534'], '1712.08155': ['arXiv:nlin/0604022', 'arXiv:1706.02873', 'arXiv:1501.01955', 'arXiv:1008.1575'], '1907.08286': ['arXiv:1711.07866'], '2001.01916': ['arXiv:1603.04467', 'arXiv:1511.06435', 'arXiv:1512.01274', 'arXiv:1802.05799'], '2003.04995': ['arXiv:1801.10612'], '2002.06684': ['arXiv:1602.02672', 'arXiv:1805.08776', 'arXiv:1703.04908', 'arXiv:1812.09755'], '1902.08185': ['arXiv:1507.07956', 'arXiv:1301.0905', 'arXiv:1502.05314', 'arXiv:1809.09635', 'arXiv:1812.02714', 'arXiv:1509.05644', 'arXiv:1509.06676

In [10]:
# Save the set of citations
with open(Config.citations_db,'wb') as file:
    pickle.dump(citations_db,file)

#### We can now download the articles. To do this we will tweak a bit the script download_to_text.py

In [13]:
def get_id_url(citation):
    idx=citation[citation.rfind(':')+1:].split('v')[0]
    control=idx.rfind('/')
    if control!=-1:
        idx=idx[control+1:]
    url='http://export.'+citation.replace(':','.org/pdf/')+'.pdf'
    return idx,url.lower()

if not os.path.exists(Config.tmp): #create directory to temporarily store pdfs if not present aready
    os.makedirs(Config.tmp)

timeout=10 #waiting seconds before stopping the download
already_have = set(os.listdir(Config.txt_db)) #getting list of papers that are already present in the directory  

num_to_add=0
num_added=0
with open('citations_db','rb') as file:
    citations_db=pickle.load(file)
    
print('Starting the script')

for citations in citations_db.values():
    
    for citation in citations:
        idx,pdf_url=get_id_url(citation)
        txt=idx+'.txt'
        pdf=idx+'.pdf'
        pdf_path=os.path.join(Config.tmp,pdf)
        txt_path=os.path.join(Config.txt_db,txt)
        try:
            if not txt in already_have:
                num_to_add+=1
                req = urlopen(pdf_url, None, timeout)
                print('Getting article %s' % (pdf_url))
                with open(pdf_path, 'wb') as file:
                    shutil.copyfileobj(req, file)
                #converting the pdf into txt needs pdftotext on the system to run
                cmd = "pdftotext %s %s" % (pdf_path, txt_path)
                exit=os.system(cmd)
                #remove the pdf to save space
                os.system('rm %s'%(pdf_path))
                num_added+=1
                #check that everything went well
                if exit!=0:
                    print('It seems like there was an error in converting %s. Please try again later. Exit status %i.'%(pdf,exit))
                    #remove the article in case the file was created
                    if os.path.isfile(txt_path):
                        os.system('rm '+txt_path)
                    num_added-=1
            
            else:
                print('%s already exists, skipping.' % (idx))
    
        except Exception as e:
            print('An error incurred while downloading: %s .'%(pdf_url))
            print(e)
print('Downloaded %i articles out of %i.'%(num_added,num_to_add))    

Starting the script
1710.01673 already exists, skipping.
1801.09003 already exists, skipping.
1901.04385 already exists, skipping.
1412.6980 already exists, skipping.
1902.04367 already exists, skipping.
1905.09474 already exists, skipping.
1809.10716 already exists, skipping.
1905.04852 already exists, skipping.
1801.06416 already exists, skipping.
1904.12442 already exists, skipping.
1905.05371 already exists, skipping.
1708.08796 already exists, skipping.
1812.08486 already exists, skipping.
1904.08351 already exists, skipping.
1702.06579 already exists, skipping.
1809.10612 already exists, skipping.
1801.07200 already exists, skipping.
1812.11143 already exists, skipping.
0812.5023 already exists, skipping.
1607.03470 already exists, skipping.
1703.09312 already exists, skipping.
0509602 already exists, skipping.
1603.04467 already exists, skipping.
1412.6980 already exists, skipping.
1701.01687 already exists, skipping.
1711.02488 already exists, skipping.
1711.00591 already exist

Getting article http://export.arxiv.org/pdf/gr-qc/0410093.pdf
It seems like there was an error in converting 0410093.pdf. Please try again later. Exit status 256.
1910.09883 already exists, skipping.
1709.05653 already exists, skipping.
0708.0034 already exists, skipping.
1902.01430 already exists, skipping.
1812.09742 already exists, skipping.
1706.04158 already exists, skipping.
1511.01071 already exists, skipping.
1701.01040 already exists, skipping.
1611.01087 already exists, skipping.
1406.4513 already exists, skipping.
1704.03578 already exists, skipping.
1203.6543 already exists, skipping.
1707.06453 already exists, skipping.
0807.3243 already exists, skipping.
1201.4330 already exists, skipping.
1204.2314 already exists, skipping.
1206.4740 already exists, skipping.
1602.02410 already exists, skipping.
1606.04934 already exists, skipping.
1701.08155 already exists, skipping.
An error incurred while downloading: http://export.arxiv.org/pdf/hep-ph/0801.pdf .
HTTP Error 404: Not F

An error incurred while downloading: http://export.arxiv.org/pdf/180506084.pdf .
HTTP Error 404: Not Found
An error incurred while downloading: http://export.arxiv.org/pdf/181211699.pdf .
HTTP Error 404: Not Found
1812.10226 already exists, skipping.
1803.07177 already exists, skipping.
1903.03427 already exists, skipping.
1807.00652 already exists, skipping.
1412.6980 already exists, skipping.
1801.07791 already exists, skipping.
1612.00593 already exists, skipping.
1802.09987 already exists, skipping.
1804.01654 already exists, skipping.
1607.07680 already exists, skipping.
1912.04312 already exists, skipping.
1601.03532 already exists, skipping.
0706.0622 already exists, skipping.
1411.2909 already exists, skipping.
1501.02188 already exists, skipping.
1701.00037 already exists, skipping.
1411.2030 already exists, skipping.
1807.08313 already exists, skipping.
1904.06553 already exists, skipping.
0404008 already exists, skipping.
0405125 already exists, skipping.
9204099 already exi

An error incurred while downloading: http://export.arxiv.org/pdf/1909.org/pdf/13837.pdf .
HTTP Error 404: Not Found
An error incurred while downloading: http://export.arxiv.org/pdf/1909.org/pdf/13837.pdf .
HTTP Error 404: Not Found
1702.07779 already exists, skipping.
1803.10829 already exists, skipping.
1810.06933 already exists, skipping.
1804.04794 already exists, skipping.
1604.00460 already exists, skipping.
1611.02675 already exists, skipping.
1105.1827 already exists, skipping.
1507.01988 already exists, skipping.
1404.2761 already exists, skipping.
1407.0334 already exists, skipping.
1811.04942 already exists, skipping.
1901.09925 already exists, skipping.
0408151 already exists, skipping.
1809.05523 already exists, skipping.
2001.05109 already exists, skipping.
1911.04709 already exists, skipping.
1912.08379 already exists, skipping.
1910.11343 already exists, skipping.
2001.10275 already exists, skipping.
1808.06670 already exists, skipping.
1502.03167 already exists, skippin

An error incurred while downloading: http://export.arxiv.org/pdf/1907.pdf .
HTTP Error 404: Not Found
1611.05243 already exists, skipping.
1611.09806 already exists, skipping.
1908.03969 already exists, skipping.
1909.07253 already exists, skipping.
1910.01395 already exists, skipping.
1811.11479 already exists, skipping.
1812.01336 already exists, skipping.
1804.08367 already exists, skipping.
1804.08367 already exists, skipping.
1804.08367 already exists, skipping.
1612.07618 already exists, skipping.
1810.10971 already exists, skipping.
1410.3394 already exists, skipping.
1905.00728 already exists, skipping.
1905.00711 already exists, skipping.
1809.09466 already exists, skipping.
1602.04946 already exists, skipping.
1610.05131 already exists, skipping.
1605.01936 already exists, skipping.
1207.7214 already exists, skipping.
1207.7235 already exists, skipping.
1303.4571 already exists, skipping.
1503.07589 already exists, skipping.
1706.09936 already exists, skipping.
0207010 alread

...

Getting article http://export.arxiv.org/pdf/1903.03107.pdf
It seems like there was an error in converting 1903.03107.pdf. Please try again later. Exit status 256.
1705.09792 already exists, skipping.
1412.6980 already exists, skipping.
1609.03499 already exists, skipping.
1907.06405 already exists, skipping.
1602.02326 already exists, skipping.
1604.02019 already exists, skipping.
1411.4317 already exists, skipping.
1405.7033 already exists, skipping.
1409.1556 already exists, skipping.
1611.01578 already exists, skipping.
1412.6980 already exists, skipping.
1502.03167 already exists, skipping.
1802.01548 already exists, skipping.
1903.03893 already exists, skipping.
1711.04528 already exists, skipping.
1711.00436 already exists, skipping.
1605.07648 already exists, skipping.
1312.4400 already exists, skipping.
1605.07146 already exists, skipping.
1912.02771 already exists, skipping.
1905.13409 already exists, skipping.
1811.03728 already exists, skipping.
2002.08313 already exists, sk

An error incurred while downloading: http://export.arxiv.org/pdf/190510866 2019.pdf .
HTTP Error 404: Not Found
An error incurred while downloading: http://export.arxiv.org/pdf/170902432 2017.pdf .
HTTP Error 404: Not Found
An error incurred while downloading: http://export.arxiv.org/pdf/170100160 2016.pdf .
HTTP Error 404: Not Found
An error incurred while downloading: http://export.arxiv.org/pdf/14126980 2014.pdf .
HTTP Error 404: Not Found
An error incurred while downloading: http://export.arxiv.org/pdf/13126114.pdf .
HTTP Error 404: Not Found
An error incurred while downloading: http://export.arxiv.org/pdf/171208708 2017.pdf .
HTTP Error 404: Not Found
An error incurred while downloading: http://export.arxiv.org/pdf/181208373 2018.pdf .
HTTP Error 404: Not Found
An error incurred while downloading: http://export.arxiv.org/pdf/180409269.pdf .
HTTP Error 404: Not Found
An error incurred while downloading: http://export.arxiv.org/pdf/190604029 2019.pdf .
HTTP Error 404: Not Found
An e

An error incurred while downloading: http://export.arxiv.org/pdf/0701186.pdf .
HTTP Error 404: Not Found
1710.07970 already exists, skipping.
1811.02240 already exists, skipping.
1804.03328 already exists, skipping.
1909.00219 already exists, skipping.
1601.05504 already exists, skipping.
1904.10880 already exists, skipping.
1906.11250 already exists, skipping.
1912.11106 already exists, skipping.
2003.01722 already exists, skipping.
1911.09768 already exists, skipping.
1812.01544 already exists, skipping.
1706.05394 already exists, skipping.
1611.01838 already exists, skipping.
1806.00900 already exists, skipping.
1706.02677 already exists, skipping.
1512.03385 already exists, skipping.
1502.01852 already exists, skipping.
1802.05300 already exists, skipping.
1502.03167 already exists, skipping.
1712.05055 already exists, skipping.
1609.04836 already exists, skipping.
1802.06175 already exists, skipping.
1708.02862 already exists, skipping.
1712.09203 already exists, skipping.
1411.77

It seems like there was an error in converting 1011.3176.pdf. Please try again later. Exit status 256.
1909.00381 already exists, skipping.
1906.07432 already exists, skipping.
1806.04011 already exists, skipping.
1607.03452 already exists, skipping.
1705.04851 already exists, skipping.
1907.11476 already exists, skipping.
1905.12648 already exists, skipping.
1906.04870 already exists, skipping.
1511.03575 already exists, skipping.
1610.05492 already exists, skipping.
1608.06879 already exists, skipping.
1905.02637 already exists, skipping.
1910.05857 already exists, skipping.
1712.00232 already exists, skipping.
1909.11774 already exists, skipping.
1903.07266 already exists, skipping.
1903.05012 already exists, skipping.
1702.08734 already exists, skipping.
1609.07228 already exists, skipping.
1808.09239 already exists, skipping.
1906.00658 already exists, skipping.
1807.00299 already exists, skipping.
1203.2288 already exists, skipping.
1908.08269 already exists, skipping.
1611.03423

An error incurred while downloading: http://export.arxiv.org/pdf/0000.0000.pdf .
HTTP Error 404: Not Found
2002.10071 already exists, skipping.
1505.07072 already exists, skipping.
1905.13285 already exists, skipping.
1805.01648 already exists, skipping.
1906.08530 already exists, skipping.
1903.08111 already exists, skipping.
1204.3071 already exists, skipping.
1904.08219 already exists, skipping.
1910.10125 already exists, skipping.
1708.04215 already exists, skipping.
1906.05316 already exists, skipping.
1812.05119 already exists, skipping.
1901.06116 already exists, skipping.
1509.03025 already exists, skipping.
1902.07698 already exists, skipping.
1712.09941 already exists, skipping.
1711.10467 already exists, skipping.
1508.01922 already exists, skipping.
1708.03288 already exists, skipping.
1605.07051 already exists, skipping.
2002.03937 already exists, skipping.
1111.0444 already exists, skipping.
1506.08056 already exists, skipping.
0212095 already exists, skipping.
1404.1578 

#### Now the strategy to evaluate the model is simple. We create two vectorised datasets, one with the citing articles and one with the cited articles. We then perform a cosine similarty search on the citing articles over the cited articles and we consider how many citations we are able to catch in the first N results. First we therefore need to vectorize the citing and the cited articles.

In [14]:
#Not all the articles in citations_db were downloaded, either for problems in the search of citations or because
#the link was not valid. so we create a new database with the papers that were actually searched
d={}
for cited, citations in citations_db.items():
    txt_labels=[]
    for citation in citations:
        idx,dummy=get_id_url(citation)
        article_path=os.path.join(Config.txt_db,idx+'.txt')
        if os.path.isfile(article_path):
            txt_labels.append(idx)
    if len(txt_labels)>0:
        d[cited]=txt_labels

In [15]:
# Get the set of all cited articles
txt_cited=[]
for citations in d.values():
    txt_cited=txt_cited+citations
txt_cited=set(txt_cited)

In [16]:
# Vectorize the cited articles
corpus=read_clean(txt_cited)
Xctd=tfidf.transform(corpus)

In [17]:
cited_vectorized={'Xctd':Xctd,'articles':np.array(list(txt_cited))}

In [18]:
# Save the vectorized form of the cited articles
with open('cited_vectorized','wb') as file:
    pickle.dump(cited_vectorized,file)

In [19]:
# Vectorize the citing articles 
corpus=read_clean(d.keys())
Xctn=tfidf.transform(corpus)

In [20]:
citing_vectorized={'Xctn':Xctn,'articles':np.array(list(d.keys()))}

In [21]:
#Save the vectorized form of the citing articles
with open('citing_vectorized','wb') as file:
    pickle.dump(citing_vectorized,file)

#### We can now write a function that performs the evaluation. This function simply computes the cosine similarity between the citing articles and the cited articles and finds how many of the cited articles are returned in the first how_many articles. Since for each citing articles the number of cited articles is not always the same, as final resut we will consider the average of the rates of correctly guessed papers.

In [22]:
#Building a function that evaluate the model
def evaluation_reccomender(cited_vectorized,citing_vectorized,d,how_many):
    Xctd=cited_vectorized['Xctd']
    articles_ctd=cited_vectorized['articles']
    Xctn=citing_vectorized['Xctn']
    articles_ctn=citing_vectorized['articles']
    similar_articles=find_similar(Xctd,Xctn,how_many)
    n_similars=0
    for i,similar in enumerate(similar_articles):
        n_similar=set(articles_ctd[similar]).intersection(set(d[articles_ctn[i]]))
        n_similars=n_similars+len(n_similar)/len(d[articles_ctn[i]])
    return n_similars/(i+1)

In [24]:
how_many=50
print("The ratio of correctly guessed papers over a window of {} articles is {:.3f}".format(how_many,evaluation_reccomender(cited_vectorized,citing_vectorized,d,how_many)))

The ratio of correctly guessed papers over a window of 50 articles is 0.006


#### As can be seen the fraction of correctly guessed articles is 0.5% over 50 articles. This result is definitly not exciting, but we need to consider that only a fraction of the cited papers can be found on the arXiv. It would be interesting to consider what results we would get with more available resources. A possible improvement of the reccomender system would be to use an approach similar to the collaborative filtering, but also in this case we would need access to more resources. 